<a href="https://colab.research.google.com/github/AndreyBocharnikov/Recommendation-systems/blob/master/hw1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit
!pip install lightfm

import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

     |████████████████████████████████| 1.1MB 13.4MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3403169 sha256=bff7b8921e2781dc8514d6e60b446fb6f8b4f5516b589aeac4e782ccf3a81f35
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit
     |████████████████████████████████| 317kB 12.3MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=706127 sha256=5327eec8ac7129afb072fe51e948bb123ca6d540618c33614fad0a309625d4b7
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

dir_name = '/content/drive/My Drive/itmo/RecSys/datasets/ml-1m'

Mounted at /content/drive/


In [ ]:
import os
import math

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy.random
import os
import numpy as np
import time

##Подготовка данных и вспомогательные функции

Прочитаем данные

In [ ]:
ratings = pd.read_csv(os.path.join(dir_name, 'ratings.dat'), delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')
print(ratings.shape)
ratings.head()

(1000209, 3)


,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [ ]:
movie_info = pd.read_csv(os.path.join(dir_name, 'movies.dat'), delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')
print(movie_info.shape)
movie_info.head()

(3883, 3)


,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Преобразуем данные: для user'ов и item'ов перейдём от id к index'ам, чтобы было удобней обращаться по индексу к их bias'ам и эмбеддингам.

In [ ]:
unique_users = np.unique(ratings.iloc[:, 0])
n_unique_users = len(unique_users)
print("Unique users:", unique_users, len(unique_users))
userid_to_userindex = {}
userindex_to_userid = {}
for i, user in enumerate(unique_users):
  userid_to_userindex[user] = i
  userindex_to_userid[i] = user 

unique_movies = np.unique(ratings.iloc[:, 1])
n_unique_movies = len(unique_movies)
print("Unique items:", unique_movies, len(unique_movies))
movieid_to_movieindex = {}
movieindex_to_movieid = {}
for i, movie in enumerate(unique_movies):
  movieid_to_movieindex[movie] = i
  movieindex_to_movieid[i] = movie 

ratings['user_index'] = ratings.user_id.apply(lambda x: userid_to_userindex.get(x))
ratings['movie_index'] = ratings.movie_id.apply(lambda x: movieid_to_movieindex.get(x))
ratings.head()

Unique users: [   1    2    3 ... 6038 6039 6040] 6040
Unique items: [   1    2    3 ... 3950 3951 3952] 3706


,user_id,movie_id,rating,user_index,movie_index
0,1,1193,5,0,1104
1,1,661,3,0,639
2,1,914,3,0,853
3,1,3408,4,0,3177
4,1,2355,5,0,2162


посчитаем bias для юзеров и айтемов

In [ ]:
bias_user = np.array(ratings.groupby('user_index').rating.mean())
bias_movie = np.array(ratings.groupby('movie_index').rating.mean())
assert(len(bias_user) == n_unique_users and len(bias_movie) == n_unique_movies)
global_bias = ratings.rating.mean()

Сделаем explicit данные

In [ ]:
data = ratings.to_numpy()
X_explicit = data[:, 3:5]
y_explicit = data[:, 2]
print(X_explicit.shape, y_explicit.shape)

(1000209, 2) (1000209,)


Сделаем implicit данные

In [ ]:
ratings["implicit"] = (ratings.rating >= 4).astype('int32')
ratings.head()

,user_id,movie_id,rating,user_index,movie_index,implicit
0,1,1193,5,0,1104,1
1,1,661,3,0,639,0
2,1,914,3,0,853,0
3,1,3408,4,0,3177,1
4,1,2355,5,0,2162,1


In [ ]:
X_implicit = np.zeros((len(unique_users), len(unique_movies)))
X_implicit[ratings.iloc[:, 3], ratings.iloc[:, 4]] = ratings.implicit

История просмотров пользователя

In [ ]:
def get_user_history(user_id):
  return [movie_info[movie_info.movie_id == x].name.to_string()
          for x in ratings[(ratings.user_id == user_id) & (ratings.rating >= 4)].movie_id]

In [ ]:
get_user_history(4)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Поиск похожих фильмов


In [ ]:
def get_similars(movie_id, movie_embeddings, k=10, mode="dot"):
  movie_index = movieid_to_movieindex.get(movie_id)
  if mode == "dot":
    similar_indexes = np.dot(movie_embeddings, movie_embeddings[movie_index]).argsort()[-k:][::-1]
  else:
    similarity = np.dot(movie_embeddings, movie_embeddings[movie_index]) / np.sqrt((movie_embeddings * movie_embeddings).sum(axis=1)) / np.sqrt((movie_embeddings[movie_index] ** 2).sum())
    similar_indexes = similarity.argsort()[-k:][::-1]
  similar_names = [movie_info[movie_info["movie_id"] == movieindex_to_movieid.get(index)]["name"].to_string() for index in similar_indexes]
  return similar_names

Рекомендации для пользователя

In [ ]:
def make_recommendations(user_id, user_embeddings, movie_embeddings, k=10, mode="dot"):
  user_index = userid_to_userindex.get(user_id)
  if mode == "dot":
    recommendation_indexes = np.dot(movie_embeddings, user_embeddings[user_index]).argsort()[-k:][::-1]
  else:
    similarity = np.dot(movie_embeddings, user_embeddings[user_index]) / np.sqrt((movie_embeddings * movie_embeddings).sum(axis=1)) / np.sqrt((user_embeddings[user_index] ** 2).sum())
    recommendation_indexes = similarity.argsort()[-k:][::-1]
  similar_names = [movie_info[movie_info["movie_id"] == movieindex_to_movieid.get(index)]["name"].to_string() for index in recommendation_indexes]
  return similar_names

##Алгоритмы

###SVD разложение на explicit данных

Напишем loss

In [ ]:
def rmse_loss(predictions, ground_truth):
  return np.sqrt(((predictions - ground_truth) ** 2).mean())

Напишем модель

In [ ]:
class SVD():
  def xavier_initialization(self, shape):
    _, d = shape
    bounde = 1 / d
    return np.random.uniform(-bounde, bounde, shape)

  def __init__(self, user_bias, item_bias, global_bias, n_users, n_items, embed_size=128, learning_rate=0.001, lr_decay=1, gamma=0):
    """
    Инициализация модели: эмбединги инициализируются с помощью xavier инициализации, bias'ы средними значениями.
    """
    self.user_embed = self.xavier_initialization((n_users, embed_size))
    self.item_embed = self.xavier_initialization((n_items, embed_size))
    self.user_bias = user_bias.copy()
    self.item_bias = item_bias.copy()
    self.global_bias = global_bias
    self.learning_rate = learning_rate
    self.lr_decay = lr_decay
    self.gamma = gamma
    self.cnt_users = np.zeros(n_users)
    self.cnt_items = np.zeros(n_items)

  def embeding_lookup(self, X):
    return self.user_embed[X[:, 0]], self.user_bias[X[:, 0]], \
           self.item_embed[X[:, 1]], self.item_bias[X[:, 1]]

  def learning_step(self, X_batch, y_batch):
    """
    Происходит 1 шаг градиентного спуска: считаются предсказания по примерам из батча, согласно им обновляются параметры и
    считается текущий loss.
    """
    predictions = self.predict(X_batch)
    self.update_params(X_batch, predictions, y_batch)
    return rmse_loss(predictions, y_batch)

  def predict(self, X_batch):
    """
    Берутся эмбединги для пользователей и item'ов из текущего батча и делаются предсказания для них.
    """
    user_embeddings, user_bias, item_embeddings, item_bias = self.embeding_lookup(X_batch)
    return np.sum(user_embeddings * item_embeddings, axis=1) + user_bias + item_bias + self.global_bias

  def update_params(self, X_batch, predictions, ground_truth):
    self.cnt_users = np.zeros_like(self.cnt_users)
    self.cnt_items = np.zeros_like(self.cnt_items)

    self.update_user(X_batch[:, 0], X_batch[:, 1], predictions, ground_truth)
    self.update_item(X_batch[:, 0], X_batch[:, 1], predictions, ground_truth)
    
  def update_user(self, user_indexes, item_indexes, predictions, ground_truth):
    """
    Считается производная для юзеров по эмбедингам и bias'у и обновляются параметры модели.
    """
    np.add.at(self.cnt_users, user_indexes, 1)
    
    gradient_embed = self.learning_rate * ((predictions - ground_truth)[:, None] * self.item_embed[item_indexes] + self.gamma * self.user_embed[user_indexes])
    gradient_embed /= self.cnt_users[user_indexes][:, None]
    np.subtract.at(self.user_embed, user_indexes, gradient_embed)

    gradient_bias = self.learning_rate * (predictions - ground_truth + self.gamma * self.user_bias[user_indexes])
    gradient_bias /= self.cnt_users[user_indexes]
    np.subtract.at(self.user_bias, user_indexes, gradient_bias)

  def update_item(self, user_indexes, item_indexes, predictions, ground_truth):
    """
    Считается производная для item'ов по эмбедингам и bias'у и обновляются параметры модели.
    """
    np.add.at(self.cnt_items, item_indexes, 1)
    
    gradient_embed = self.learning_rate * ((predictions - ground_truth)[:, None] * self.user_embed[user_indexes] + self.gamma * self.item_embed[item_indexes])
    gradient_embed /= self.cnt_items[item_indexes][:, None]
    np.subtract.at(self.item_embed, item_indexes, gradient_embed)

    gradient_bias = self.learning_rate * (predictions - ground_truth + self.gamma * self.item_bias[item_indexes])
    gradient_bias /= self.cnt_items[item_indexes]
    np.subtract.at(self.item_bias, item_indexes, gradient_bias)
  
  def update_lr(self):
    self.learning_rate *= self.lr_decay

Обучим модель, разобьём данные на тренировочные и тестовые как 0.8 и 0.2, чтобы знать что мы не переобучаемся

In [ ]:
model = SVD(bias_user, bias_movie, global_bias, n_unique_users, n_unique_movies, learning_rate=0.005, lr_decay=0.99, embed_size=64, gamma=0.1)
batch_size = 16
n = X_explicit.shape[0]
p = np.random.permutation(n)
X_explicit, y_explicit = X_explicit[p], y_explicit[p]
X_train, X_test = X_explicit[:math.ceil(n * 0.8)], X_explicit[math.ceil(n * 0.8):] # разбивает X_explicit на train и test
y_train, y_test = y_explicit[:math.ceil(n * 0.8)], y_explicit[math.ceil(n * 0.8):] # разбивает y_explicit на train и test
n_batches_train = (X_train.shape[0] + batch_size - 1) // batch_size
  
epoches = 100 # кол-во эпох
for epoch in range(epoches):
  batches_loss = []
  for i in range(n_batches_train):
    X_batch, y_batch = X_train[i * batch_size : (i + 1) * batch_size], y_train[i * batch_size : (i + 1) * batch_size] # текущий батч
    loss = model.learning_step(X_batch, y_batch) # делаем 1 шаг градиентного спуска
    batches_loss.append(loss)
  
  model.update_lr() # после каждой эпохи уменьшаем lr
  if epoch % 10 == 0 or epoch == epoches - 1:
    predictions = model.predict(X_test) 
    rmse = rmse_loss(predictions, y_test) # считаем loss на тесте  
    print("Epoch number =", epoch)
    print("Rmse on train:", np.mean(batches_loss))
    print("Rmse on test:", rmse)
    print("=============")

Epoch number = 0
Rmse on train: 2.8066177920654085
Rmse on test: 1.7603730076505744
Epoch number = 10
Rmse on train: 0.926194099335473
Rmse on test: 0.9507247294418009
Epoch number = 20
Rmse on train: 0.8836704838611382
Rmse on test: 0.9108449168358468
Epoch number = 30
Rmse on train: 0.8707509823739459
Rmse on test: 0.8990872709805094
Epoch number = 40
Rmse on train: 0.86362331879451
Rmse on test: 0.8932291632662728
Epoch number = 50
Rmse on train: 0.8576413547245135
Rmse on test: 0.8888294546761891
Epoch number = 60
Rmse on train: 0.851935998157206
Rmse on test: 0.8849146216922728
Epoch number = 70
Rmse on train: 0.8468138133826649
Rmse on test: 0.8815877024701477
Epoch number = 80
Rmse on train: 0.8426042724993952
Rmse on test: 0.879005876834848
Epoch number = 90
Rmse on train: 0.8392260386326517
Rmse on test: 0.8770462515996386
Epoch number = 99
Rmse on train: 0.8367297026325137
Rmse on test: 0.8756688824927151


Переобучения особо нет, обучим модель на всех данных используя те же гиперпараметры

In [ ]:
model = SVD(bias_user, bias_movie, global_bias, n_unique_users, n_unique_movies, learning_rate=0.005, lr_decay=0.99, embed_size=64, gamma=0.1)
n_batches = (X_explicit.shape[0] + batch_size - 1) // batch_size

for epoch in range(epoches):
  batches_loss = []
  for i in range(n_batches):
    X_batch, y_batch = X_explicit[i * batch_size : (i + 1) * batch_size], y_explicit[i * batch_size : (i + 1) * batch_size]
    loss = model.learning_step(X_batch, y_batch)
    batches_loss.append(loss)
  if epoch % 10 == 0 or epoch == epoches - 1:
    print(epoch, np.mean(batches_loss))

0 2.5758649291871785
10 0.9040907943093638
20 0.8742413395848392
30 0.8638414612024338
40 0.8538798364089341
50 0.845276702693424
60 0.8394722497171836
70 0.835383214030834
80 0.8320209481965833
90 0.8290379220293516
99 0.8267012456840654


Найдём похожие фильмы

In [ ]:
get_similars(1, model.item_embed, mode="cos")

['0    Toy Story (1995)',
 '584    Aladdin (1992)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '591    Beauty and the Beast (1991)',
 '2728    Big (1988)',
 '2012    Little Mermaid, The (1989)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '33    Babe (1995)',
 '2618    Tarzan (1999)']

Сделаем рекомендации

In [ ]:
make_recommendations(4, model.user_embed, model.item_embed)

['3346    Mirror, The (Zerkalo) (1975)',
 '1461    Love and Other Catastrophes (1996)',
 '1147    Mina Tannenbaum (1994)',
 '3747    Other Side of Sunday, The (S�ndagsengler) (1996)',
 '2563    Saragossa Manuscript, The (Rekopis znaleziony ...',
 '3746    Official Story, The (La Historia Oficial) (1985)',
 '104    Nobody Loves Me (Keiner liebt mich) (1994)',
 '2761    Cabaret Balkan (Bure Baruta) (1998)',
 '3576    Cleo From 5 to 7 (Cl�o de 5 � 7) (1962)',
 '2169    Seven Beauties (Pasqualino Settebellezze) (1976)']

###ALS разложение на implicit данных

Напишем взвешенных loss, который штрафует сильнее за предсказания 0 на месте 1.

In [ ]:
def weighted_rmse(predictions, ground_truth, alpha):
  weights = 1 + alpha * ground_truth
  return np.sqrt((weights * (predictions - ground_truth) ** 2).mean())

In [ ]:
class ALS():
  def xavier_initialization(self, shape):
    _, d = shape
    bounde = 1 / d
    return np.random.uniform(-bounde, bounde, shape)
  
  def __init__(self, data, embed_size, n_steps=10, alpha=100, gamma=0.25):
    self.n_users, self.n_items = data.shape
    self.user_embed = self.xavier_initialization((self.n_users, embed_size))
    self.item_embed = self.xavier_initialization((self.n_items, embed_size))
    self.data = data
    self.c = 1 + alpha * data
    self.n_steps = n_steps
    self.alpha = alpha
    self.gamma = gamma
    self.eye = np.eye(embed_size)
    self.eye_items = np.eye(self.n_items)
    self.eye_users = np.eye(self.n_users)
  
  def fit(self):
    """
    Обучение модели: @self.n_steps раз делается обновление параметров согласно @update_params.
    """
    for i in range(self.n_steps):
      predictions = self.predict()
      start = time.time()
      self.update_params()
      print("Step", i, "loss =", weighted_rmse(predictions, self.data, self.alpha), "time took", time.time() - start)

  def predict(self):
    """
    Предсказания делаются на основе скалярного произведения.
    """
    return np.dot(self.user_embed, self.item_embed.T)
  
  def update_params(self):
    """
    Обновления весов описанное в статье http://yifanhu.net/PUB/cf.pdf.
    """
    tmp_item = np.dot(self.item_embed.T, self.item_embed)
    for user in range(self.n_users):
      nonzero = (self.c[user] - 1).astype(bool) # ускорение матричного произведения, благодаря которому оно работает за O(f^2 * n_u), а не O(f^2 * n), где n_u - кол-во не нулевых элементов.
      iTi = self.alpha * self.item_embed.T[:, nonzero] @ self.item_embed[nonzero]
      self.user_embed[user] = np.linalg.inv(tmp_item + iTi + self.gamma * self.eye) @ self.item_embed.T @ (self.c[user] * self.data[user])
    
    tmp_user = np.dot(self.user_embed.T, self.user_embed)
    for item in range(self.n_items):
      nonzero = (self.c[:, item] - 1).astype(bool)
      uTu = self.alpha * self.user_embed.T[:, nonzero] @ self.user_embed[nonzero]
      self.item_embed[item] = np.linalg.inv(tmp_user + uTu + self.gamma * self.eye) @ self.user_embed.T @ (self.c[:, item] * self.data[:, item])

In [ ]:
model = ALS(X_implicit, n_steps=10, alpha=10, embed_size=64, gamma=0.25)
model.fit()

Step 0 loss = 0.5316992137675064 time took 13.28004789352417
Step 1 loss = 0.30439773104354234 time took 13.593106746673584
Step 2 loss = 0.2541990150861876 time took 13.603131771087646
Step 3 loss = 0.2473237645661728 time took 13.743133306503296
Step 4 loss = 0.24468871035175582 time took 14.520085334777832
Step 5 loss = 0.24337373539891827 time took 13.922136068344116
Step 6 loss = 0.2426129692543914 time took 13.81650447845459
Step 7 loss = 0.24212919544706354 time took 14.088450908660889
Step 8 loss = 0.24180123436320294 time took 13.97995924949646
Step 9 loss = 0.2415687019347711 time took 13.868547201156616


Найдём похожие фильмы

In [ ]:
get_similars(1, model.item_embed)

['3045    Toy Story 2 (1999)',
 '0    Toy Story (1995)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '1245    Groundhog Day (1993)',
 '591    Beauty and the Beast (1991)',
 '360    Lion King, The (1994)',
 '2252    Pleasantville (1998)',
 "1854    There's Something About Mary (1998)"]

Сделаем рекомендации

In [ ]:
make_recommendations(4, model.user_embed, model.item_embed)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '1220    Terminator, The (1984)',
 '1180    Raiders of the Lost Ark (1981)',
 '1196    Alien (1979)',
 '1366    Jaws (1975)',
 '585    Terminator 2: Judgment Day (1991)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1023    Die Hard (1988)']

###BPR на implicit данных

Сделаем массив set'ов id фильмов, которые понравились user'ам, чтобы при negative sampling'е быстро понимать отрицательный ли данных пример.

In [ ]:
users_pos_samples = []
for i, user in enumerate(X_implicit):
  indexes_pos = set(np.where(user == 1)[0])
  users_pos_samples.append(indexes_pos)

Сделаем dataset, размер которого равен кол-во отрицательных примеров, умноженное на суммарное кол-во фильмов которые понравились всем пользователям.

In [ ]:
class BPRDataset(Dataset):
  def __init__(self, users_pos_samples, n_items, n_negative_samples=3):
    self.users_pos_samples = users_pos_samples
    self.n_items = n_items
    self.n_users = len(users_pos_samples)
    self.data = [] # массив пар (user_index, movie_index), где movie_index - положительный пример для user_index
    for user_index, pos_samples in enumerate(users_pos_samples):
      for pos_sample in pos_samples:
        self.data.append([user_index, pos_sample])
    self.n_negative_samples = n_negative_samples
    
  def __len__(self):
    return self.n_negative_samples * len(self.data)

  def __getitem__(self, id):
    user, pos_sample = self.data[id // self.n_negative_samples]
    while True:
      neg_sample = np.random.randint(0, self.n_items) # random negative sampling
      if neg_sample not in self.users_pos_samples[user]: # если пример действительно негативный, то тройка найдена
        break
    return (user, pos_sample, neg_sample)

In [ ]:
batch_size = 64
bpr_dataset = BPRDataset(users_pos_samples, n_unique_movies, n_negative_samples=10)
bpr_dataloader = DataLoader(bpr_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

Сделаем модель, которая возвращает пару - score эмбединга пользователя с эмбидингом положительным и отрицательным примерами.

In [ ]:
class DotModel(nn.Module):
  def __init__(self, n_users, n_items, embed_size):
    super(DotModel, self).__init__()
    self.user_embed = nn.Embedding(n_users, embed_size)
    self.item_embed = nn.Embedding(n_items, embed_size)
    
    sqrt_embed_size = np.sqrt(embed_size)
    self.user_embed.weight.data.uniform_(-1 / sqrt_embed_size, 1 / sqrt_embed_size)
    self.item_embed.weight.data.uniform_(-1 / sqrt_embed_size, 1 / sqrt_embed_size)
    
  
  def forward(self, users, pos_samples, neg_samples):
    user_embed = self.user_embed(users)
    pos_items = self.item_embed(pos_samples)
    neg_items = self.item_embed(neg_samples)
    return (torch.sum(user_embed * pos_items, axis=1), torch.sum(user_embed * neg_items, axis=1))

Loss из оригинальной статьи, где в качестве \hat{x}_{uij} используется разность похожести эмбединга пользователя с эмбедингом положительного примера и с эмбедингом отрицательного примера.  
Вместо максимизации правдоподобия делается минимизация минус правдоподобия.

In [ ]:
def bpr_loss(pos_dots, neg_dots):
  return -torch.log(torch.sigmoid(pos_dots - neg_dots)).mean()

Функция обучения заданной модели на заданном dataloader'е.

In [ ]:
def train_model(model, dataloader, n_epoches=3, lr=0.001, weight_decay=0, verbose_every=10000):
  opt = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  
  for epoch in range(n_epoches):
    batch_loss = []
    start = time.time()
    for i, batch in enumerate(dataloader):
      opt.zero_grad()
      users, pos_samples, neg_samples = batch
      pos_dots, neg_dots = model(users, pos_samples, neg_samples)
      loss = bpr_loss(pos_dots, neg_dots)
      loss.backward()
      opt.step()
      batch_loss.append(loss.item())
      if i % verbose_every == 0:
        print("epoch =", epoch, "batch =", i, "loss =", np.mean(batch_loss), "time took =", time.time() - start)
        start = time.time()
        batch_loss = []

Обучим модель на BPR данных

In [ ]:
model = DotModel(n_unique_users, n_unique_movies, 64)
train_model(model, bpr_dataloader, verbose_every=10000)

epoch = 0 batch = 0 loss = 0.6939931511878967 time took = 0.631885290145874
epoch = 0 batch = 10000 loss = 0.40493274340629576 time took = 70.85172295570374
epoch = 0 batch = 20000 loss = 0.21643283905759453 time took = 72.04491329193115
epoch = 0 batch = 30000 loss = 0.17922773022688926 time took = 71.4583694934845
epoch = 0 batch = 40000 loss = 0.15973469309434293 time took = 69.06842970848083
epoch = 0 batch = 50000 loss = 0.14473877490162848 time took = 70.42462277412415
epoch = 0 batch = 60000 loss = 0.13368620417378843 time took = 69.87066149711609
epoch = 0 batch = 70000 loss = 0.12680612712372094 time took = 70.45058274269104
epoch = 0 batch = 80000 loss = 0.1185579035161063 time took = 69.58873796463013
epoch = 1 batch = 0 loss = 0.12277768552303314 time took = 0.6208419799804688
epoch = 1 batch = 10000 loss = 0.10490929673984647 time took = 74.9869704246521
epoch = 1 batch = 20000 loss = 0.10007667351877317 time took = 76.2795901298523
epoch = 1 batch = 30000 loss = 0.0980128

Посмотрим на похожие фильмы и рекомендации

In [ ]:
get_similars(1, model.item_embed.weight.data)

['0    Toy Story (1995)',
 '584    Aladdin (1992)',
 '360    Lion King, The (1994)',
 '591    Beauty and the Beast (1991)',
 '1526    Hercules (1997)',
 "2286    Bug's Life, A (1998)",
 '2012    Little Mermaid, The (1989)',
 '773    Hunchback of Notre Dame, The (1996)',
 '3045    Toy Story 2 (1999)',
 '590    Snow White and the Seven Dwarfs (1937)']

In [ ]:
make_recommendations(4, model.user_embed.weight.data, model.item_embed.weight.data)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '847    Godfather, The (1972)',
 '1180    Raiders of the Lost Ark (1981)',
 '1196    Alien (1979)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1366    Jaws (1975)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1182    Aliens (1986)',
 '2878    Goldfinger (1964)']

### WARP на implicit данных

Напишем другой датасет, модель оставим такую же, потому что изменяется только то, на каких примерах мы её учим.  
В датасете будем делать 1 negative sample, но не рандомный, а тот на котором модель работает плохо - предсказывает негативный пример более похожим чем позитивный, установим ограничение на время поиска - max_samples, чтобы можно было учить больше эпох, это улучшает общее качество модели.

In [ ]:
class WARPDataset(Dataset):
  def __init__(self, users_pos_samples, n_items, user_embed, item_embed, max_samples=100):
    self.users_pos_samples = users_pos_samples
    self.n_items = n_items
    self.n_users = len(users_pos_samples)
    self.user_embed = user_embed
    self.item_embed = item_embed
    self.max_samples = max_samples
    self.data = [] # аналогично BPR
    for user_index, pos_samples in enumerate(users_pos_samples):
      for pos_sample in pos_samples:
        self.data.append([user_index, pos_sample])
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, id):
    user, pos_sample = self.data[id]
    cnt = 0 # счётчик кол-ва негативный примеров, которые сравнивались с позитивным примером pos_sample
    user_embed = self.user_embed(torch.Tensor([user]).long())
    pos_embed = self.item_embed(torch.Tensor([pos_sample]).long())
    while True:
      neg_sample = np.random.randint(0, self.n_items) # random negative sampling 
      if neg_sample not in self.users_pos_samples[user]:
        cnt += 1
        neg_score = self.item_embed(torch.Tensor([neg_sample]).long())
        if torch.sum(user_embed * pos_embed) < torch.sum(user_embed * neg_score) or cnt == self.max_samples: # проверка на то, что модель работает на этой паре плохо
          break
    return (user, pos_sample, neg_sample)

In [ ]:
model = DotModel(n_unique_users, n_unique_movies, 64)

warp_dataset = WARPDataset(users_pos_samples, n_unique_movies, model.user_embed, model.item_embed)
warp_dataloader = DataLoader(warp_dataset, batch_size=64, shuffle=True, num_workers=4)

train_model(model, warp_dataloader, n_epoches=20, verbose_every=4000)

epoch = 0 batch = 0 loss = 0.707985520362854 time took = 0.2880270481109619
epoch = 0 batch = 4000 loss = 0.5802115963697434 time took = 95.40622210502625
epoch = 0 batch = 8000 loss = 0.2926865451782942 time took = 94.91562247276306
epoch = 1 batch = 0 loss = 1.0930906534194946 time took = 0.6948308944702148
epoch = 1 batch = 4000 loss = 0.5926576810181141 time took = 422.8968026638031
epoch = 1 batch = 8000 loss = 0.48400610926747323 time took = 423.36377692222595
epoch = 2 batch = 0 loss = 0.9128168225288391 time took = 0.5136561393737793
epoch = 2 batch = 4000 loss = 0.4557212028503418 time took = 281.3136899471283
epoch = 2 batch = 8000 loss = 0.34209805617854 time took = 287.29824328422546
epoch = 3 batch = 0 loss = 1.0808762311935425 time took = 0.8863968849182129
epoch = 3 batch = 4000 loss = 0.5296302679032088 time took = 464.7663915157318
epoch = 3 batch = 8000 loss = 0.39570797718688844 time took = 461.7753691673279
epoch = 4 batch = 0 loss = 0.9408712387084961 time took = 0

In [ ]:
get_similars(1, model.item_embed.weight.data)

['0    Toy Story (1995)',
 '591    Beauty and the Beast (1991)',
 '2692    Iron Giant, The (1999)',
 '2502    Matrix, The (1999)',
 '1526    Hercules (1997)',
 '2225    Antz (1998)',
 '3045    Toy Story 2 (1999)',
 "523    Schindler's List (1993)",
 '584    Aladdin (1992)',
 '1262    Fantasia (1940)']

In [ ]:
make_recommendations(4, model.user_embed.weight.data, model.item_embed.weight.data)

['2878    Goldfinger (1964)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '3633    Mad Max (1979)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '1196    Alien (1979)',
 '2882    Fistful of Dollars, A (1964)',
 '2502    Matrix, The (1999)',
 '2916    Robocop (1987)',
 '2875    Dirty Dozen, The (1967)',
 '257    Star Wars: Episode IV - A New Hope (1977)']